In [2]:
from datasets import load_dataset,Dataset
START_SYM="<--start-->"

data_files = ["000_00000.parquet","001_00000.parquet","002_00000.parquet",]

fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb/sample/10BT", data_files=data_files,split="train",num_proc=6,cache_dir='/data/bert-chunker-v2/dataset/cache')



In [5]:
fw2=fw.filter(lambda x: x['token_count']>50 and x['language_score']>0.95,num_proc=6)
print(fw2)

Filter (num_proc=6): 100%|██████████| 3143000/3143000 [00:11<00:00, 278468.14 examples/s]

Dataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'],
    num_rows: 1405557
})


In [ ]:
print(fw2[2001]['text'])

In [13]:
fw2 = fw2.select(range(2000))

## 数据合成 generating dataset：

In [ ]:
from dataclasses import dataclass, field
import json
import time
import logging
import os
from typing import Dict, Optional, List, Union
import torch
from torch.utils.data import Dataset
import re
import torch
from torch import nn
from torch.utils.data import DataLoader
import transformers
from transformers import Trainer, GPTQConfig, AutoModel, AutoTokenizer, AutoConfig
from transformers.trainer_pt_utils import LabelSmoother
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from accelerate.utils import DistributedType
from datasets import load_dataset
import safetensors.torch


IGNORE_TOKEN_ID = -100 
START_SYM = "<--start-->"


def tokenize_function(item, tokenizer, max_len, stride):

    # chunks=re.split(START_SYM,item['input'])
    chunks=re.split("\n",item['text'])

    input_ids=[]
    labels=[]
    for _,c in enumerate(chunks):
        c=tokenizer(c,truncation=False)['input_ids']
        #去掉头尾，最后两端再加上special tokens在头尾
        c=c[1:-1]
        if c:
            input_ids+=c

            cl=len(c)*[0]

            cl[0]=1
            labels+=cl
    #判断数据是否是空，抛出异常
    if not labels:
        print('--------')
        print(item)
    labels[0]=0

    full_texts = {
            "input_ids": [],
            "labels": [],
            "attention_mask": [],
        }
    for start_idx in range(0, len(input_ids), stride):
        end_idx = start_idx + max_len  - 2

        if end_idx>len(input_ids) +50 and full_texts["input_ids"]:
            break

        window_input_ids = input_ids[start_idx:end_idx]
        window_labels = labels[start_idx:end_idx]

        # 加入bert 头尾
        window_input_ids = [tokenizer.cls_token_id]+window_input_ids+[tokenizer.sep_token_id]

        window_labels=[IGNORE_TOKEN_ID]+window_labels+[IGNORE_TOKEN_ID]

        # 填充到max_len
        len_now = len(window_input_ids)
        window_input_ids = window_input_ids + [0] * (max_len - len_now)
        window_labels = window_labels + [IGNORE_TOKEN_ID] * (max_len - len_now)
        attention_mask = [1] * len_now + [0] * (max_len - len_now)

        full_texts["input_ids"].append(window_input_ids)
        full_texts["labels"].append(window_labels)
        full_texts["attention_mask"].append(attention_mask)


    return full_texts

model_max_length = 255

tokenizer = AutoTokenizer.from_pretrained(
    "/data/all-MiniLM-L6-v2",
    padding_side="right",
    model_max_length=model_max_length,
    trust_remote_code=True,
)


train_dataset = fw2.map(
    tokenize_function,
    remove_columns=fw.column_names,
    fn_kwargs={"tokenizer": tokenizer, "max_len": model_max_length, "stride": 50},
    num_proc=6,
)

In [18]:
train_dataset

Dataset({
    features: ['input_ids', 'labels', 'attention_mask'],
    num_rows: 2000
})

In [19]:
train_dataset = train_dataset.to_pandas()


In [ ]:
train_dataset = train_dataset.explode(['input_ids',"labels","attention_mask"],ignore_index=True)
print(train_dataset)


In [21]:
train_dataset.to_parquet("/data/bert-chunker-v2/dataset/fwtrain-slidewindow.parquet")